In [9]:
print 1./3

0.333333333333


In [10]:
import numpy as np
from ROOT import TH1F,TCanvas, TGraph, TF1, TFitResult,kGreen, kRed,kBlue, TH1D,gStyle, gROOT ,TGraphErrors,TFitResultPtr,TFitResult
from math import *
N_total = 0
N0 = 1e9
A = 0.35
taumu0 = 2.2e-6
gamma = 29.3
phase = 0.3
lifetime = 2.2e-6
n = 0.142
beta = 1
w_a = 2.*np.pi*(230000.)
R0 = 7112 
p_magic = 3.094
bnldist = 1 #1 for bnl distribution 
gamma_magic = 29.3

def p(r):
    r_magic = 7112 
    p_magic = 3.094 
    n = 0.142 # or 0.122
    beta = 1
    gamma_magic = 29.3
    lifetime_magic = 2.2e-6 # as tau_0 = lifetime 
    tau_magic = gamma_magic*lifetime_magic # tau as dilated time 
    p = (((1-n)*p_magic*(r - r_magic))/r_magic) + p_magic
    gamma = ((p - p_magic)/p_magic)*(beta**2)*gamma_magic + gamma_magic
    tau = gamma*lifetime_magic
    deltaw_a = -2*w_a*(beta**(2))*n*(1-n)*(((r-R0)/R0)**(2)) 
    return p,gamma,tau,deltaw_a

def spread(Numb,step_r):
    gamma_s    = []
    momentum_s = []
    ef_co      = []
    gen = np.linspace(-(step_r/2),(step_r/2),Numb)
    radius_s = [7112]*(Numb) + gen
    for k in range(0,Numb):
        gamma_s.append(p(radius_s[k])[1])
        momentum_s.append(p(radius_s[k])[0])
        #ef_co.append(p(radius_s[k])[3]) 
        ef_co.append(0) 
    return radius_s,gamma_s,momentum_s,ef_co

def BNL(r):
    if (r >= 7094 and r < 7098 ) or (r>= 7133 and r< 7139):
        return 0.1
    if (r >= 7098 and r < 7100 ) or (r>= 7130 and r< 7133):
        return 0.2
    if (r >= 7100 and r < 7101 ) or (r>= 7129 and r< 7130):
        return 0.3
    if (r >= 7101 and r < 7102 ) or (r>= 7127 and r< 7129):
        return 0.4
    if (r >= 7102 and r < 7103 ) or (r>= 7124 and r< 7127):
        return 0.5
    if (r >= 7103 and r < 7105 ) or (r>= 7122 and r< 7124):
        return 0.6
    if (r >= 7105 and r < 7106 ) or (r>= 7120 and r< 7122):
        return 0.7
    if (r >= 7106 and r < 7107 ) or (r>= 7116 and r< 7120):
        return 0.8
    if (r >= 7107 and r < 7108 ) or (r>= 7114 and r< 7116):
        return 0.9
    if (r >= 7108 and r < 7114 ):
        return 1
    else :
        return 0

def weight(Num,radius_s,bnl):
    weight_s = []
    normalization = 0
    total = 0
    tot = []
    if bnl == 0:
        print 'hello'
        weight_s = [1./3]*Num
        total = np.sum(weight_s)
        tot = [total]*Num #changi
        weight_spread = np.divide(weight_s,tot)
        normalization = np.sum(weight_s)
        return weight_s,normalization
    elif bnl == 1:
        for j in range(len(radius_s)):
            weight_s.append(BNL(radius_s[j]))
        weight_s = np.array(weight_s)
        total = np.sum(weight_s)
        tot = [total]*Num
        weight_s = np.divide(weight_s,tot)
        normalization = np.sum(weight_s)  
        return weight_s,normalization

def fivepar(t,gamma,phase,efcor):
    y = N0*np.exp(-t/ (taumu0*gamma))*(1 + A*np.cos((w_a + efcor)*t + phase))
    return y

def data_maker(nbin,weight_s,gamma_s,phase_s,ef_co):
    t    = np.linspace(0,750e-6,nbin)
    t_er = np.linspace(0,0,nbin)
    data = []
    data_er = []
    Ntotal = 0
    for i in range(len(t)):
        element = 0
        for k in range(0,len(gamma_s)):
            element += weight_s[k]*fivepar(t[i],gamma_s[k],phase_s[k],ef_co[k]) 
        data.append(element)
    Ntotal = np.sum(data)
    data_er = np.sqrt(data)
    print Ntotal,'Ntotal'
    return t,t_er,Ntotal,data,data_er
def noise_maker(ydata,nbin):
    data = []
    rand = np.random.normal(0,1,nbin)
    for k in range(len(ydata)):
        data.append(ydata[k] + np.sqrt(ydata[k])*rand[k])
    print 'Ntotal might be this number',np.sum(data)
    return data

def fitter(xdata,ydata,xer,yer,N0,gamma_s,Asym,w_a,phase_s,weight_s,momentum_s):
    deltap = momentum_s[1]-momentum_s[0] 
    p1 = momentum_s[0] 
    xdata = np.array(xdata)
    ydata = np.array(ydata)
    dummy = np.array(xer)
    yerror = np.array(yer)
    c = TCanvas()
    g = TGraphErrors(xdata.size, xdata.astype(np.double),ydata.astype(np.double), dummy.astype(np.double), yerror.astype(np.double))
    #g1.GetXaxis().SetRange(0,100000)
    fitfunc = TF1('fitfunc', \
    '[0]*[5]*exp(-x/ (2.2e-6*((( -1*[8]*[1])/[9])+[1])))*(1 + [2]*cos([3]*x+  [4])) + \
     [0]*[6]*exp(-x/ (2.2e-6*[1])                      )*(1 + [2]*cos([3]*x + [4])) +  \
     [0]*[7]*exp(-x/ (2.2e-6*(((  1*[8]*[1])/[9])+[1])))*(1 + [2]*cos([3]*x + [4]))',0,750e-6)
# 0 -> Nzero 1->gamma  2->asymmetry 3->w_a 4->phase 5,6,7->weights 8->deltap 9->pmagic
    fitfunc.SetNpx(5000)
    fitfunc.SetParameter(0,N0*1.3) 
    #fitfunc.SetParLimits(0,N0*0.7,N0*1.6)
    fitfunc.SetParameter(1,gamma_magic*1.2) 
    #fitfunc.SetParLimits(1,gamma_magic*0.5,gamma_magic*1.5)
    fitfunc.SetParameter(2,0.3) #Asym
    fitfunc.SetParameter(3,w_a*(1+30e-6)) #omega_a
    #fitfunc.SetParLimits(3,w_a*(1-30e-6),w_a*(1+30e-6))
    fitfunc.SetParameter(4,np.pi/6)
    #fitfunc.SetParLimits(4,0,2*np.pi)
    for i in range(5,8):
        fitfunc.FixParameter(i,weight_spread[i-5]) 
    fitfunc.FixParameter(8,deltap)
    fitfunc.FixParameter(9,p_magic)
    
    fit = g.Fit('fitfunc','S')  
    
    p0 = fit.Get().Parameter(0)
    p1 = fit.Get().Parameter(1)
    p2 = fit.Get().Parameter(2)
    p3 = fit.Get().Parameter(3)
    p4 = fit.Get().Parameter(4)
    e0 = fit.Get().ParError(0)
    e1 = fit.Get().ParError(1)
    e2 = fit.Get().ParError(2)
    e3 = fit.Get().ParError(3)
    e4 = fit.Get().ParError(4)
    
    print 'fit results on w_a',p3,'and error from the fit',(e3/p3)*(10**(6)),'ppm'
    NDF = fitfunc.GetNDF()
    CHI2 = fitfunc.GetChisquare()
    PPM  = ((p3-1445132.62065)/1445132.62065)*(10**(6))
    #print 'the extra parameter',p28,'the error',e28
    #print 'sigma of w_a',((2*np.pi*e3)/w_a)*1e6,'ppm'
    #print 'what I believe is the sigma of w_A',e3*1e6,'ppm'
    print PPM,'ppm'
    print ((w_a-p3)/w_a)*1e6,'ppm'
    print CHI2/NDF,'chi2'
    return CHI2/NDF,PPM,p0,p1,p2,p4

#S.P.R.E.A.D
delta_r = 30
ON = 3 #spread number
radius_spread,gamma_spread,momentum_spread,ef_cor = spread(ON,delta_r)
weight_spread,normal = weight(ON,radius_spread,0)
phase_spread = [-0.3,0,0.3]
print 'Normalization of the weights:',normal
bins = 5000
x = []
y1 = []
x_er = []
y_er = []
y = []
x,x_er,N_total,y1,y_er = data_maker(bins,weight_spread,gamma_spread,phase_spread,ef_cor) 
sigma = []
ppm = []
Nzero = []
GAMMA = []
ASYM = []
PHASE = []
NUMB = 1000
for k in range(NUMB):
    wut = 0
    y = []
    y = noise_maker(y1,bins)   
    wut = fitter(x,y,x_er,y_er,N0,gamma_spread,A,w_a,phase_spread,weight_spread,momentum_spread)
    sigma.append(wut[0])
    ppm.append(wut[1])
    Nzero.append(wut[2])
    GAMMA.append(wut[3])   
    ASYM.append(wut[4])
    PHASE.append(wut[5])             
                  
                  
                  
                  
    


hello
Normalization of the weights: 1.0
430330447480.66077 Ntotal
Ntotal might be this number 430330001093.5485
fit results on w_a 1445138.29864 and error from the fit 0.06731490209 ppm
3.92904372869 ppm
-3.92904282581 ppm
0.989313258465 chi2
Ntotal might be this number 430331238513.8998
fit results on w_a 1445138.447 and error from the fit 0.0673145575793 ppm
4.03170823644 ppm
-4.03170733355 ppm
0.975381515854 chi2
Ntotal might be this number 430331876409.14526
fit results on w_a 1445138.36529 and error from the fit 0.0673135716008 ppm
3.97516290371 ppm
-3.97516200082 ppm
1.0102233016 chi2
Ntotal might be this number 430330713152.2829
fit results on w_a 1445138.23715 and error from the fit 0.0673147734061 ppm
3.88649248167 ppm
-3.88649157879 ppm
0.990805550874 chi2
Ntotal might be this number 430330491204.4483
fit results on w_a 1445138.49759 and error from the fit 0.0673155136653 ppm
4.06671046592 ppm
-4.06670956304 ppm
0.972104427947 chi2
Ntotal might be this number 430329454271.439

Ntotal might be this number 430329920949.6791
fit results on w_a 1445138.34307 and error from the fit 0.0673141071675 ppm
3.95979097311 ppm
-3.95979007023 ppm
0.997172286546 chi2
Ntotal might be this number 430330351054.06335
fit results on w_a 1445138.3117 and error from the fit 0.0673136699157 ppm
3.93808455037 ppm
-3.93808364749 ppm
1.01425476002 chi2
Ntotal might be this number 430330062260.29315
fit results on w_a 1445138.18448 and error from the fit 0.0673153318873 ppm
3.85004930148 ppm
-3.8500483986 ppm
1.04923880892 chi2
Ntotal might be this number 430331021753.25793
fit results on w_a 1445138.34459 and error from the fit 0.0673149780827 ppm
3.9608428217 ppm
-3.96084191881 ppm
0.972243016783 chi2
Ntotal might be this number 430330656879.4122
fit results on w_a 1445138.43549 and error from the fit 0.0673142456088 ppm
4.02373879564 ppm
-4.02373789275 ppm
1.00219825982 chi2
Ntotal might be this number 430330871868.8165
fit results on w_a 1445138.40587 and error from the fit 0.0673

Ntotal might be this number 430330882092.45
fit results on w_a 1445138.27595 and error from the fit 0.0673140160006 ppm
3.91334383105 ppm
-3.91334292816 ppm
1.02089251261 chi2
Ntotal might be this number 430330776276.3434
fit results on w_a 1445138.66597 and error from the fit 0.0673146237509 ppm
4.18322647779 ppm
-4.1832255749 ppm
1.03005636016 chi2
Ntotal might be this number 430330256371.4014
fit results on w_a 1445138.42472 and error from the fit 0.0673145534496 ppm
4.01629061368 ppm
-4.01628971079 ppm
1.00530773978 chi2
Ntotal might be this number 430330660438.1688
fit results on w_a 1445138.25399 and error from the fit 0.0673141246047 ppm
3.89814667191 ppm
-3.89814576902 ppm
1.00983660223 chi2
Ntotal might be this number 430330600790.5595
fit results on w_a 1445138.2591 and error from the fit 0.0673140117497 ppm
3.90168031829 ppm
-3.90167941541 ppm
1.02370015003 chi2
Ntotal might be this number 430329747407.3023
fit results on w_a 1445138.24471 and error from the fit 0.0673144891

fit results on w_a 1445138.40857 and error from the fit 0.0673151937313 ppm
4.0051100914 ppm
-4.00510918851 ppm
1.0142196418 chi2
Ntotal might be this number 430330055774.2585
fit results on w_a 1445138.2355 and error from the fit 0.0673144439054 ppm
3.88535473339 ppm
-3.88535383051 ppm
1.02827802842 chi2
Ntotal might be this number 430329901838.377
fit results on w_a 1445138.2869 and error from the fit 0.0673144238723 ppm
3.92091963954 ppm
-3.92091873665 ppm
0.994850464294 chi2
Ntotal might be this number 430329163346.07745
fit results on w_a 1445138.48192 and error from the fit 0.0673139549397 ppm
4.05587328947 ppm
-4.05587238659 ppm
0.988832626858 chi2
Ntotal might be this number 430330425996.28625
fit results on w_a 1445138.10884 and error from the fit 0.0673151886297 ppm
3.79770774963 ppm
-3.79770684675 ppm
1.01604958061 chi2
Ntotal might be this number 430330055756.33167
fit results on w_a 1445138.37413 and error from the fit 0.0673121558381 ppm
3.98127854885 ppm
-3.98127764596 p

fit results on w_a 1445138.31011 and error from the fit 0.0673147636629 ppm
3.93698154562 ppm
-3.93698064273 ppm
0.960931158835 chi2
Ntotal might be this number 430331123098.6279
fit results on w_a 1445138.5046 and error from the fit 0.0673128522825 ppm
4.07156450003 ppm
-4.07156359714 ppm
1.01562439355 chi2
Ntotal might be this number 430329712788.082
fit results on w_a 1445138.24487 and error from the fit 0.0673140744739 ppm
3.89183551628 ppm
-3.89183461339 ppm
1.02266259306 chi2
Ntotal might be this number 430330487174.7437
fit results on w_a 1445138.47969 and error from the fit 0.0673134789925 ppm
4.05432485467 ppm
-4.05432395179 ppm
1.00281623186 chi2
Ntotal might be this number 430330133572.24774
fit results on w_a 1445138.46983 and error from the fit 0.0673142164277 ppm
4.04750693528 ppm
-4.04750603239 ppm
0.989998153115 chi2
Ntotal might be this number 430329916612.88605
fit results on w_a 1445138.41113 and error from the fit 0.067314863815 ppm
4.00688230569 ppm
-4.00688140281 

...

Ntotal might be this number 430330226502.428
fit results on w_a 1445138.19921 and error from the fit 0.0673146104516 ppm
3.86024401504 ppm
-3.86024311215 ppm
0.991800112522 chi2
Ntotal might be this number 430331335466.2976
fit results on w_a 1445138.27155 and error from the fit 0.0673150234735 ppm
3.91029719654 ppm
-3.91029629365 ppm
1.02029915415 chi2
Ntotal might be this number 430330678845.6074
fit results on w_a 1445138.2755 and error from the fit 0.0673147931892 ppm
3.91303380353 ppm
-3.91303290064 ppm
0.973045885481 chi2
Ntotal might be this number 430329653143.5114
fit results on w_a 1445138.23912 and error from the fit 0.0673145743768 ppm
3.8878597799 ppm
-3.88785887701 ppm
1.01335360127 chi2
Ntotal might be this number 430330757252.88983
fit results on w_a 1445138.44704 and error from the fit 0.0673149985591 ppm
4.03173667107 ppm
-4.03173576818 ppm
0.989865768609 chi2
Ntotal might be this number 430329880433.7254
fit results on w_a 1445138.40742 and error from the fit 0.06731

Ntotal might be this number 430330272711.7948
fit results on w_a 1445138.22924 and error from the fit 0.0673130620277 ppm
3.88102108392 ppm
-3.88102018104 ppm
0.983178283233 chi2
Ntotal might be this number 430329566396.3542
fit results on w_a 1445138.24122 and error from the fit 0.0673139306039 ppm
3.88931350917 ppm
-3.88931260629 ppm
0.993039988474 chi2
Ntotal might be this number 430330439776.5615
fit results on w_a 1445138.37919 and error from the fit 0.0673156039516 ppm
3.98478633133 ppm
-3.98478542844 ppm
1.03581444225 chi2
Ntotal might be this number 430329682199.82074
fit results on w_a 1445138.31486 and error from the fit 0.0673145815914 ppm
3.94026673665 ppm
-3.94026583377 ppm
0.974347079375 chi2
Ntotal might be this number 430331077112.1938
fit results on w_a 1445138.23232 and error from the fit 0.067314221636 ppm
3.88315552223 ppm
-3.88315461934 ppm
0.992585304549 chi2
Ntotal might be this number 430330858591.6026
fit results on w_a 1445138.26895 and error from the fit 0.06

fit results on w_a 1445138.03649 and error from the fit 0.0673135582294 ppm
3.74764198838 ppm
-3.74764108549 ppm
0.971981384918 chi2
Ntotal might be this number 430329637652.0438
fit results on w_a 1445138.3994 and error from the fit 0.0673146530242 ppm
3.99877059635 ppm
-3.99876969347 ppm
0.973549760522 chi2
Ntotal might be this number 430330547298.1098
fit results on w_a 1445138.14781 and error from the fit 0.067314934565 ppm
3.82467008975 ppm
-3.82466918686 ppm
0.985085495486 chi2
Ntotal might be this number 430330188019.63544
fit results on w_a 1445138.25091 and error from the fit 0.0673156481318 ppm
3.89601439091 ppm
-3.89601348803 ppm
0.991964832349 chi2
Ntotal might be this number 430330835222.29266
fit results on w_a 1445138.21548 and error from the fit 0.0673145187924 ppm
3.87149733057 ppm
-3.87149642769 ppm
0.991156586364 chi2
Ntotal might be this number 430330213903.2304


KeyboardInterrupt: 

 FCN=4941.62 FROM MIGRAD    STATUS=CONVERGED     155 CALLS         156 TOTAL
                     EDM=9.68589e-08    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   0.8 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           9.99999e+08   2.15610e+03   2.05967e+02  -1.65107e-07
   2  p1           2.93000e+01   4.47082e-05  -1.01743e-05  -7.97904e+00
   3  p2           3.39579e-01   2.05998e-06  -1.57084e-08   2.33772e+02
   4  p3           1.44514e+06   9.72793e-02   1.48195e-02  -1.03294e-03
   5  p4           7.53162e-06   8.86218e-06  -7.88526e-07   1.93690e+01
   6  p5           3.33333e-01     fixed    
   7  p6           3.33333e-01     fixed    
   8  p7           3.33333e-01     fixed    
   9  p8           5.59896e-03     fixed    
  10  p9           3.09400e+00     fixed    
 FCN=4872.03 FROM MIGRAD    STATUS=CONVERGED     155 CALLS         156 TOTAL
                   

In [ ]:
print gamma_spread,radius_spread,ef_cor,weight_spread

In [ ]:
print min(sigma),max(sigma)

In [ ]:
sigma = np.array(sigma)
c = TCanvas()
h = TH1F('#chi^{2}/d.o.f','#chi^{2}/d.o.f for N_{0}=1e9 ',15,0.90,1.10)
for i in range(NUMB):
    h.Fill(sigma[i])
h.SetFillColorAlpha(kBlue, 0.35);
#g.GetXaxis().SetTitle('gamma')
h.GetYaxis().SetTitle('#chi^{2}/d.o.f')
c.Draw()
h.Draw()

In [ ]:
print min(ppm),max(ppm)

In [ ]:
ppm = np.array(ppm)
c = TCanvas()
h = TH1F('#delta #omega_{a}/#omega_{a}','#delta #omega_{a}/#omega_{a} in ppm for N_{0}=1e9',15,-0.25,0.25)
for i in range(NUMB):
    h.Fill(ppm[i])
h.SetFillColorAlpha(kBlue, 0.35);
h.GetYaxis().SetTitle('ppm')
c.Draw()
h.Draw()


In [ ]:
c = TCanvas()
h = TH1F("Nzero","Nzero*1.3",20,min(Nzero),max(Nzero))
for k in range(NUMB):
    h.Fill(Nzero[k] )
h.SetFillColorAlpha(kBlue, 0.35);
c.Draw()
h.Draw() 

In [ ]:
c = TCanvas()
h = TH1F("gamma","gamma",20,min(GAMMA),max(GAMMA))
for k in range(NUMB):
    h.Fill(GAMMA[k] )
h.SetFillColorAlpha(kBlue, 0.35);
c.Draw()
h.Draw() 

In [ ]:
c = TCanvas()
h = TH1F("Asymetry","Asymetry",20,min(ASYM),max(ASYM))
for k in range(NUMB):
    h.Fill(ASYM[k] )
h.SetFillColorAlpha(kBlue, 0.35);
c.Draw()
h.Draw() 

In [ ]:
c = TCanvas()
h = TH1F("Phase","Phase",20,min(PHASE),max(PHASE))
for k in range(NUMB):
    h.Fill(PHASE[k] )
h.SetFillColorAlpha(kBlue, 0.35);
c.Draw()
h.Draw() 